<a href="https://colab.research.google.com/github/freguti/ProgettoDataSpaces/blob/main/Progetto_DataSpaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Analysis of "Predicting Pulsar Star"**

Simone Soncin - s263094


##**Index**

##**Introduction**



In [2]:
#capture removes the cell's output
%%capture
import pandas as pd
import numpy as np
from google.colab import drive
!pip install -U imbalanced-learn
from imblearn.over_sampling import ADASYN
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from IPython.display import HTML
!pip install nbconvert
!pip install -q gwpy

In [3]:
%%capture
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g64pM7YS2Q_Z6OQGSgWF9kY78xmjpXdxcO8Dt4ujsyfY5a_AH_VVyk


In [4]:
#QUI INSERIRÒ TUTTE LE UTILITY PER I GRAFICI TODO: rimuovere commento
def oneFeatureHistogram(dataset,feature,title) :
  x = dataset[feature]
  image, plot = plt.subplots(figsize=(16, 10))
  plot.hist(x)
  for rect in plot.patches:
    height = rect.get_height()
    if height > 0:
      plot.annotate(f'{int(height)}', xy=(rect.get_x()+rect.get_width()/2, height), 
                xytext=(0, 5), textcoords='offset points', ha='center', va='bottom') 
  plt.xlabel(feature)
  plt.ylabel('value')
  plt.title(title)
  plt.show()

In [5]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [6]:
dataset = pd.read_csv('/content/drive/My Drive/DataSpaces/pulsar_stars.csv')
print("il dataset ha %d righe e %d colonne" % dataset.shape)

il dataset ha 17898 righe e 9 colonne


Procedo al conteggio dei campi nulli all'interno del dataset, suddividendoli per colonna

In [7]:
print(dataset.isnull().sum())

 Mean of the integrated profile                  0
 Standard deviation of the integrated profile    0
 Excess kurtosis of the integrated profile       0
 Skewness of the integrated profile              0
 Mean of the DM-SNR curve                        0
 Standard deviation of the DM-SNR curve          0
 Excess kurtosis of the DM-SNR curve             0
 Skewness of the DM-SNR curve                    0
target_class                                     0
dtype: int64


In [8]:
X, y = make_classification(n_classes = 2, )
target_class = dataset['target_class'].value_counts()
print("1: è una stella\n0: non è una stella\n" + target_class.to_string())

1: è una stella
0: non è una stella
0    16259
1     1639


Il dataset è abbastanza sbilanciato, bisognerà normalizzare i pesi dei record.
I record classificati come "0" dovranno avere un peso di circa 1/10 rispetto a quelli classificati come "1".

Bisogna fare l'oversampling solo sul test set

In [9]:
oneFeatureHistogram(dataset,"target_class","Distribuzione classi")

ImportError: ignored

<Figure size 1152x720 with 1 Axes>

In [10]:
%%capture
'''
SERVE PER CONVERTIRE UN NOTEBOOK IN HTML
TODO: da scommentare nella build finale
https://stackoverflow.com/questions/53460051/convert-ipynb-notebook-to-html-in-google-colab


%%shell
jupyter nbconvert --to html /content/Progetto_DataSpaces.ipynb
''' 